In [3]:
import json, csv, os, time

import pandas as pd
import urllib
import tweepy

In [4]:
# Load credentials from json file
def get_credentials(file: str):
    with open(file, "r") as f:
        creds = json.load(f)
        return creds

def write_data(credentials, hashtag:str, runtime:int, language = 'en'):
    creds = get_credentials(credentials)
    auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
    auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
 
    api = tweepy.API(auth,wait_on_rate_limit=True) 
    
    file_name = f"{hashtag}_tweet_data.csv"
    csvFile = open(os.path.join('Data',file_name), 'w')
    csvWriter = csv.writer(csvFile)
    
    for i, tweet in enumerate(tweepy.Cursor(api.search, q=f"#{hashtag}", since = '2020-08-01', lang="en").items()):
        if i != runtime*60:
            if (not tweet.retweeted) and ('RT @' not in tweet.text):
                csvWriter.writerow([tweet.created_at, tweet.id_str.encode('utf-8'), tweet.text.encode('utf-8'), [i['text'].encode('utf-8').lower() for i in tweet.entities['hashtags']], tweet.location.encode('utf-8')])
        else:
            break
    if os.path.exists(os.path.join('Data',file_name)):
        print(f"Tweets added successfully to {file_name}")

In [ ]:
credential_path = r'Data\twitter_credentials.json'

write_data(credential_path, 'biden', 5)
write_data(credential_path, 'trump', 5)
write_data(credential_path, 'theright', 5)
write_data(credential_path, 'theleft', 5)

In [15]:
display(pd.read_csv(r'Data\theright_tweet_data.csv'))

,2020-10-05 17:45:47,b'1313173553517342720',b'How to upset the #TheRight? Tell a lie.\nHow to upset #TheLeft? Tell the truth.',"[b'theright', b'theleft']"
0,2020-10-05 14:29:20,b'1313124113825837056',b'\xe2\x80\x9cPlease make me understand how yo...,[]
1,2020-10-04 18:56:01,b'1312828842080337922',b'@realDonaldTrump please legalize marijuana #...,"[b'asap', b'thankyou', b'cometogether']"
2,2020-10-04 17:02:32,b'1312800282317815809',b'@RyanTreeFiddy @VirtualAnnette @jonostrower ...,[b'theright']
3,2020-10-04 11:09:15,b'1312711376276779008',b'@aslohand @onesmoker420 @hrenee80 @realDonal...,"[b'theleft', b'grit']"
4,2020-10-03 20:38:08,b'1312492151209521153',b'@JackPosobiec @Mom3Law The whole left did?? ...,[]
5,2020-10-03 08:26:22,b'1312307998291886081',b'@Bubblebathgirl @9NEWSNANCY #TheFarLeft comp...,[b'thefarleft']
6,2020-10-02 15:35:05,b'1312053497752424454',"b'Matchup between the two most evil, darkest, ...",[]
7,2020-10-01 18:15:44,b'1311731539210887170',b'OurEyesUponTheFather\n\n#LIONofJUDAH #BRIDEo...,"[b'lionofjudah', b'brideofchrist', b'themessiah']"
8,2020-10-01 14:37:16,b'1311676560127258625',"b""@darrengrimes_ Aren't you in #Prison yet ? #...","[b'prison', b'askingfor', b'people', b'ruin']"
9,2020-09-30 20:29:18,b'1311402764036079619',"b""She's right \xf0\x9f\x91\x87 #JoeBiden, need...","[b'joebiden', b'dump45']"
